<a href="https://colab.research.google.com/github/franklinwillemen/Sherlock-Hemlock-Graph-Based-NER/blob/main/data_build_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Create a symbolic link to the Colab Notebooks directory
nb_path = '/content/notebooks'

# Check if the symbolic link or directory already exists
if not os.path.exists(nb_path):
    os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)

# Add the symbolic link to the system path
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [1]:
import logging
import transformers as t
import datasets
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd
import spacy

c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("joelito/mapa")
df = pd.DataFrame(dataset["train"].to_pandas())

Found cached dataset json (C:/Users/Frank/.cache/huggingface/datasets/joelito___json/joelito--mapa-b7771d6b2e7c8e8e/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 3/3 [00:00<00:00, 334.31it/s]


German, English, Spanish, French, Italian, Dutch, and Portuguese.

In [4]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download nl_core_news_sm
!python -m spacy download pt_core_news_sm

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB 1.3 MB/s eta 0:00:12
     --------------------------------------- 0.1/14.6 MB 648.1 kB/s eta 0:00:23
     --------------------------------------- 0.1/14.6 MB 819.2 kB/s eta 0:00:18
      --------------------------------------- 0.2/14.6 MB 1.1 MB/s eta 0:00:14
      --------------------------------------- 0.3/14.6 MB 1.5 MB/s eta 0:00:10
     - -------------------------------------- 0.5/14.6 MB 2.0 MB/s eta 0:00:08
     - -------------------------------------- 0.6/14.6 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.1 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.1 MB

In [20]:
models = ["de_core_news_sm", "en_core_web_sm", "es_core_news_sm", "fr_core_news_sm", "it_core_news_sm", "nl_core_news_sm", "pt_core_news_sm"]

In [21]:
# List of language codes to drop
languages_to_keep = ["de", "en", "es", "fr", "it", "nl", "pt"]

# Drop the rows with the specified language codes
mapa_df = df[df['language'].isin(languages_to_keep)]

# mapa_df = mapa_df.drop(["type", "file_name", "sentence_number"], axis=1)

# Reset the index of the filtered DataFrame
mapa_df.reset_index(drop=True, inplace=True)

In [22]:
def extract_pos_tags_deps(sentence, nlp):
    try:
        doc = nlp(" ".join(sentence))
        pos = [token.pos_ for token in doc]
        dep = [token.dep_ for token in doc]
    except:
        pos, dep = [], [], []
    return pos, dep

In [36]:
def process_languages(df, models):
    df['pos'] = ''
    df['dep'] = ''
    for model in models:
        lang = model.split("_")[0]
        nlp = spacy.load(model)
        # nlp.disable_pipes([pipe for pipe in nlp.pipe_names if pipe not in ["tagger", "parser"]])
        lang_df = df.loc[df['language'] == lang].copy()
        lang_df[['pos', 'dep']] = lang_df['tokens'].apply(lambda x: pd.Series(extract_pos_tags_deps(x, nlp)))
        df.loc[df['language'] == lang, ['pos', 'dep']] = lang_df[['pos', 'dep']].values
    return df

In [37]:
df = process_languages(mapa_df, models)

In [39]:
df.to_csv('mapagraph_subset.csv', index=False)

In [41]:
from datasets import Dataset
from huggingface_hub import HfApi

In [48]:
# Load your local dataset into a Dataset object
dataset = Dataset.from_csv("mapagraph_subset.csv")

# Save the dataset to Hugging Face
dataset_name = 'MAPAGraph'
dataset_description = 'An work in progress'
api_key = ""

dataset.push_to_hub(dataset_name,api_key)

Found cached dataset csv (C:/Users/Frank/.cache/huggingface/datasets/csv/default-20c3dd06f34b7fea/0.0.0)
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:05<00:00,  5.53s/it]
